# Perkenalan

=================================================

Milestone 3

Nama  : Muhammad Insani

Batch : FTDS-010-HCK

Program ini dibuat dengan tujuan melakukan validasi data menggunakan Great Expectation. Data yang akan divalidasi berasal dari hasil pembersihan data yang disimpan di postgreSQL. Setelah proses validasi, data akan dikirimkan ke ElasticSearch untuk kemudian divisualisasikan melalui dashboard Kibana.

=================================================

# Import Libaries

In [24]:
import pandas as pd
import psycopg2 as db
from great_expectations.data_context import FileDataContext

# Deskripsi Dataset

| Kolom                       | Deskripsi                                         |
|-----------------------------|------------------------------------------------------|
| BPA                         | Building Permit Application (Aplikasi Izin Mendirikan Bangunan) |
| Address                     | Alamat proyek                                         |
| Description                 | Deskripsi proyek                                      |
| PTS Existing Units          | Jumlah unit yang ada sebelum proyek                   |
| PTS Proposed Units          | Jumlah unit yang diusulkan dalam proyek               |
| Proposed Units              | Jumlah unit yang direncanakan untuk dibangun          |
| Net Units                   | Jumlah unit bersih setelah proyek selesai (PTS Proposed Units - PTS Existing Units) |
| Net Units Completed         | Jumlah unit bersih yang telah selesai dibangun         |
| First Completion Date       | Tanggal penyelesaian unit pertama                      |
| Latest Completion Date      | Tanggal penyelesaian unit terakhir                     |
| Extremely Low Income        | Jumlah unit yang diperuntukkan bagi masyarakat berpenghasilan sangat rendah |
| Very Low Income             | Jumlah unit yang diperuntukkan bagi masyarakat berpenghasilan rendah |
| Low Income                  | Jumlah unit yang diperuntukkan bagi masyarakat berpenghasilan menengah ke bawah |
| Moderate Income              | Jumlah unit yang diperuntukkan bagi masyarakat berpenghasilan menengah |
| Moderate Income - NDR        | Jumlah unit yang diperuntukkan bagi masyarakat berpenghasilan menengah dengan Non-Deed Restricted (NDR) |
| Affordable Units            | Jumlah total unit yang dianggap terjangkau            |
| Market Rate                 | Jumlah unit yang disewakan dengan harga pasar         |
| Affordable Units Estimate   | Estimasi jumlah unit yang terjangkau                   |
| Supervisor District         | Distrik pengawas yang bertanggung jawab                |
| Analysis Neighborhood       | Lingkungan analisis proyek                             |
| Planning Dist.               | Distrik perencanaan proyek                             |
| Plan Area                    | Area perencanaan proyek                                |
| Form No.                     | Nomor formulir izin                                   |
| Permit Type                 | Jenis izin yang diberikan                             |
| Issued Date                 | Tanggal izin diterbitkan                               |
| Authorization Date          | Tanggal izin disahkan                                 |
| Zoning District              | Distrik zonasi proyek                                 |
| Project Affordability Type   | Jenis keterjangkauan proyek                            |
| BlockLot                     | Nomor blok dan lot proyek                             |
| PPTS Project ID              | Project ID dalam Planning Permit Tracking System (PPTS) |


# Data Load

In [25]:
df = pd.read_csv('D:\HCK-010\phase_2\Milestone\p2-ftds010-hck-m3-muhamadinsani17\data\P2M3_muhammad_insani_data_clean.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5186 entries, 0 to 5185
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   index                       5186 non-null   int64 
 1   bpa                         5186 non-null   object
 2   address                     5186 non-null   object
 3   description                 5186 non-null   object
 4   pts_existing_units          5186 non-null   int64 
 5   pts_proposed_units          5186 non-null   int64 
 6   proposed_units              5186 non-null   int64 
 7   net_units                   5186 non-null   int64 
 8   net_units_completed         5186 non-null   int64 
 9   first_completion_date       5186 non-null   object
 10  latest_completion_date      5186 non-null   object
 11  extremely_low_income        5186 non-null   int64 
 12  very_low_income             5186 non-null   int64 
 13  low_income                  5186 non-null   int6

# Initiate Data Context
> Pada tahap ini, akan dilakukan pembuatan root directory dari project Great Expectations.

In [26]:
context = FileDataContext.create(project_root_dir='./')

# Connect to A Datasource
> Pada tahap ini akan dilakukan penyiapan data source dan data asset yang akan divaldiasi dengan Great Expectations.

In [27]:
# initializing datasource
datasource_name = 'Housing_Production'
datasource = context.sources.add_pandas(datasource_name)

asset_name = 'cleaned_data'
path_to_data = 'D:\HCK-010\phase_2\Milestone\p2-ftds010-hck-m3-muhamadinsani17\data\P2M3_muhammad_insani_data_clean.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

# Expectation Suite

## Create an Expectation Suite

In [28]:
# Creat an expectation suite
expectation_suite_name = 'expectation-Housing_Production-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,index,bpa,address,description,pts_existing_units,pts_proposed_units,proposed_units,net_units,net_units_completed,first_completion_date,...,planning_dist,plan_area,form_no,permit_type,issued_date,authorization_date,zoning_district,project_affordability_type,blocklot,ppts_project_id
0,0,201602089004,950 MARKET ST,"TO ERECT 13 STORIES, 2 BASEMENTS, TYPE I-A, 47...",0,470,470,470,242,09/03/2021 12:00:00 AM,...,4 - Downtown,Downtown,1,Site Permit,12/14/2017 12:00:00 AM,04/08/2019 12:00:00 AM,C-3-G,Inclu: Land Dedication,0342001,2013.1049
1,1,201611283577,30 OTIS ST,"TO ERECT 27 STORIES, 2 BASEMENT, TYPE I-A, 404...",0,404,404,404,395,07/19/2021 12:00:00 AM,...,8 - Mission,Downtown,1,Site Permit,01/22/2019 12:00:00 AM,08/09/2019 12:00:00 AM,C-3-G,Inclu: Fee Payment,3505016,2015-010013PRJ
2,2,201306210213,2171 03RD ST,"TO ERECT 7 STORIES,1 BASEMENT, TYPE IB, 109 UN...",0,109,109,109,104,07/15/2020 12:00:00 AM,...,9 - South of Market,Mission Bay,1,Site Permit,06/08/2017 12:00:00 AM,12/27/2017 12:00:00 AM,UMU,Inclu: On-Site BMR,4045003,2013.0784
3,3,201801128565,1856 PACIFIC AV,"PER ADU UNIT ORDINANCE# 162-16, CONVERT GROUND...",11,16,16,5,1,08/11/2020 12:00:00 AM,...,2 - Marina,Unknown,3,Full BP,07/11/2019 12:00:00 AM,07/11/2019 12:00:00 AM,RH-2,Unknown,0576011,2018-001265PRJ
4,4,9824818,1328 MISSION ST,4 STORY (12 LIVE WORK) 1 STORY PARKING,0,12,12,12,12,04/02/2016 12:00:00 AM,...,4 - Downtown,Downtown,2,Full BP,08/16/1999 12:00:00 AM,08/16/1999 12:00:00 AM,C-3-G,Inclu: On-Site BMR,3508020,1998.940


## Expectations

### 1. to be unique
>Alasan utama penggunaan validasi `to be unique` kolom "BPA" adalah untuk menjamin bahwa setiap aplikasi izin mendirikan bangunan memiliki identifikasi yang unik. Dengan menerapkan validasi `to be unique` pada kolom "BPA," dapat dihindari adanya duplikasi atau penggunaan nomor aplikasi izin yang sama untuk proyek yang berbeda. Hal ini sangat penting dalam konteks data izin mendirikan bangunan, karena setiap proyek perlu diidentifikasi secara unik agar dapat dilacak dan dikelola dengan efisien.

In [29]:
# expectation values within the bpa column are all unique
print('bpa unique values validation')
unique_bpa=validator.expect_column_values_to_be_unique('bpa')
print(f"Expectation is {unique_bpa['success']}", '\n')

bpa unique values validation


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Expectation is True 



### 2. to be between min_value and max_value
> Great Expectations `to be between min_value and max_value` digunakan untuk memvalidasi kolom-kolom pendapatan diantaranya 'extremely_low_income' dan 'moderate_income__ndr', Ekspektasi yang ditetapkan adalah bahwa nilai-nilai dalam kolom-kolom tersebut harus berada dalam rentang antara 0 hingga 200. Tujuan ekspektasi ini adalah memastikan bahwa data terkait pendapatan mencerminkan jumlah unit yang sesuai dengan kategori tertentu agar membantu memastikan konsistensi dan validitas data terkait proyek perizinan mendirikan bangunan, sesuai dengan konteks dataset yang telah diberikan sebelumnya.

In [30]:
# expectations values min and max within the income columns
for col in ['extremely_low_income', 'very_low_income', 'low_income', 'moderate_income', 'moderate_income__ndr']:
    print(col, 'expectation')
    result=validator.expect_column_values_to_be_between(
        column=col, min_value=0, max_value=200
        )
    print(f"Expectation is {result['success']}", '\n')

extremely_low_income expectation


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Expectation is True 

very_low_income expectation


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Expectation is True 

low_income expectation


...

Expectation is True 

moderate_income expectation


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Expectation is True 

moderate_income__ndr expectation


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Expectation is True 



### 3. to_be_in_set
>Dengan menggunakan ekspektasi `to_be_in_set` ini, program memastikan bahwa kolom 'project_affordability_type' dalam dataset memuat nilai-nilai yang sesuai dengan kategori-kategori keterjangkauan proyek yang telah ditentukan sebelumnya. Ini membantu meningkatkan keandalan dan kegunaan dataset dalam analisis data terkait proyek perizinan mendirikan bangunan.

In [31]:
# expectation values are in the set or contain one of the following category
print('project_affordability_type validation')
result=validator.expect_column_values_to_be_in_set('project_affordability_type',
                                            ['Inclu: Land Dedication', 'Inclu: Fee Payment', 'Inclu: On-Site BMR', 'Unknown',
                                            'Inclu: Combination', '100% Affordable','Inclu: Off-Site BMR','Inclu: Off-Site BMR, Fee',
                                            'Exempt']
)
print(f"Expectation is {result['success']}", '\n')

project_affordability_type validation


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Expectation is True 



### 4. to_be_in_type_list
>Dengan menggunakan ekspektasi `to_be_in_type_list` ini, program memastikan bahwa kolom-kolom numerik dalam dataset memiliki nilai-nilai berupa bilangan bulat, sehingga meningkatkan keandalan dan konsistensi tipe data dalam analisis data terkait proyek perizinan mendirikan bangunan.

In [32]:
# expectation values of numeric columns has to be either interger
numeric_list=['pts_existing_units', 'pts_proposed_units', 'proposed_units', 'net_units','net_units_completed','affordable_units','market_rate','supervisor_district','form_no']
for num in numeric_list:
    print(num, 'numeric type validation')
    result=validator.expect_column_values_to_be_in_type_list(num, ['int64'])
    print(f"Expectation is {result['success']}", '\n')

pts_existing_units numeric type validation


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Expectation is True 

pts_proposed_units numeric type validation


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Expectation is True 

proposed_units numeric type validation


...

Expectation is True 

supervisor_district numeric type validation


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Expectation is True 

form_no numeric type validation


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Expectation is True 



### 5. to_not_match_regex_list
>Dengan menggunakan ekspektasi `to_not_match_regex_list`, program memastikan bahwa nama wilayah analisis dan jenis izin dalam dataset tidak mengandung karakter tidak diinginkan, seperti angka atau simbol khusus. Validasi ini bertujuan mencegah ketidaksesuaian, menjaga konsistensi dan akurasi data dalam analisis proyek perizinan mendirikan bangunan, serta mengurangi potensi kesalahan atau ketidaksesuaian dalam penamaan wilayah analisis dan jenis izin.

In [33]:
# expectation that the analysis_neighborhood and permit_type names are only letters, '.', ',' and spaces
for val in ['analysis_neighborhood','permit_type']:
    print(val, 'not match regex list validation')
    pattern_val=validator.expect_column_values_to_not_match_regex_list(
    column=val,
    regex_list=[r'.*[@\\"0-9].*']
)
    print(f"Expectation is {pattern_val['success']}", '\n')

analysis_neighborhood not match regex list validation


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Expectation is True 

permit_type not match regex list validation


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Expectation is True 



### 6. table_row_count_to_be_between
>Dengan `table_row_count_to_be_between`, ekspektasi ini membantu mencegah kesalahan analisis yang dapat timbul jika dataset terlalu kecil atau terlalu besar. Dengan demikian, program memastikan bahwa dataset memiliki ukuran yang sesuai, meningkatkan keandalan data dalam analisis proyek perizinan mendirikan bangunan.

In [ ]:
# Expectation "expect_table_row_count_to_be_between"
print('data set validation table row count')
table_row_count_expectation = validator.expect_table_row_count_to_be_between(
    min_value=1000,
    max_value=10000
)

print(f"Expectation is {table_row_count_expectation['success']}", '\n')

### 7. to_be_of_type
>Ekspektasi `to_be_of_type` ini mengharapkan bahwa nilai-nilai dalam kolom description memiliki tipe data "object" (objek atau string). Validasi ini membantu memastikan konsistensi dan keamanan data, mencegah kesalahan atau ketidaksesuaian tipe data yang dapat memengaruhi analisis. 

In [35]:
# Expectation "expect_column_values_to_be_of_type"
print('description type validation')
column_type_expectation = validator.expect_column_values_to_be_of_type(
    "description",
    "object"
)
print(f"Expectation is {column_type_expectation['success']}", '\n')

description type validation


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Expectation is True 



### 8. to_not_be_null
>Ekspektasi `to_not_be_null` digubakan untuk memvalidasi kolom 'ppts_project_id' dengan mengharapkan bahwa nilai-nilai yang sebelumnya bernilai null sekarang tidak lagi null. Ekspektasi ini diterapkan untuk memastikan bahwa setiap proyek perizinan mendirikan bangunan memiliki ID yang terdefinisi dengan baik, meningkatkan konsistensi dan validitas data.

In [36]:
# expectation values that was null not to be null
print('ppts_project_id not null validation')
not_null= validator.expect_column_values_to_not_be_null('ppts_project_id')
print(f"Expectation is {not_null['success']}")

ppts_project_id not null validation


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation is True


## Menyimpan Expectation Suite

In [37]:
validator.save_expectation_suite(discard_failed_expectations=False)

# Checkpoint

In [38]:
Checkpoint = context.add_or_update_checkpoint(
    name = 'Checkpoint',
    validator = validator,
)

In [39]:
checkpoint_result = Checkpoint.run()

Calculating Metrics:   0%|          | 0/71 [00:00<?, ?it/s]

# Data Docs

In [40]:
context.build_data_docs()

{'local_site': 'file://d:\\HCK-010\\phase_2\\Milestone\\p2-ftds010-hck-m3-muhamadinsani17\\gx\\uncommitted/data_docs/local_site/index.html'}